In [ ]:
import cv2
import time
# My face detectors interface
import FaceDetectors
import numpy as np

def pointInContainer(container,x,y):
    [x1,y1,x2,y2] = container
    return x1 < x < x2 and y1 < y < y2

def mouse_events(event, x, y, flags, params):
    global px, py, pdx, pdy, lmb_mantenido, containers, active_container, active_container_p, containersT, z_index
    # Movimiento del puntero     
    if event==cv2.EVENT_MOUSEMOVE:
        if lmb_mantenido:
            pdx = x - px
            pdy = y - py
            return
    elif event==cv2.EVENT_LBUTTONDOWN:
        for i in range(0,len(containers)):
            moved_container = [containers[i][j]+containersT[i][j%2][2] for j in range(0,len(containers[i]))]
            if pointInContainer(moved_container,x,y):
                active_container = i
                active_container_p = [v for v in moved_container[:2]]
                lmb_mantenido = True
                px = x
                py = y
                pdx = pdy = 0
                z_index = [i]+[v for v in z_index if v != i]
                break
    elif event==cv2.EVENT_LBUTTONUP:
        active_container = None
        lmb_mantenido = False

def setFeatureContainer(container,xs,ys):
    container[0] = min(xs)
    container[1] = min(ys)
    container[2] = max(xs)
    container[3] = max(ys)
    return

def setAllFeatureContainers(shape):
    setFeatureContainer(mouth_container,shape[48:60,0],shape[48:60,1])
    setFeatureContainer(leye_container,shape[36:42,0],shape[36:42,1])
    setFeatureContainer(reye_container,shape[42:48,0],shape[42:48,1])
    setFeatureContainer(nose_container,shape[27:36,0],shape[27:36,1])
    return

def generateMaskAndFeature(frame,shape):
    mask = np.zeros(frame.shape[:2],dtype=np.uint8)
    cv2.fillPoly(mask,pts=[shape],color=255)
    feature = cv2.bitwise_and(frame,frame,mask=mask)
    return (mask,feature)

def generateAllMasksAndFeatures(frame,shape):
    (mouth_mask,mouth) = generateMaskAndFeature(frame,shape[48:60])
    (leye_mask,leftEye) = generateMaskAndFeature(frame,shape[36:42])
    (reye_mask,rightEye) = generateMaskAndFeature(frame,shape[42:48])
    nose_points = np.array([shape[i] for i in [27,27,31,32,33,34,35]])
    nose_offset = (nose_points[6][0]-nose_points[2][0])/4
    nose_points[0][0] = nose_points[0][0]+nose_offset
    nose_points[1][0] = nose_points[1][0]-nose_offset
    (nose_mask,nose) = generateMaskAndFeature(frame,nose_points)
    return ([mouth_mask,leye_mask,reye_mask,nose_mask],[mouth,leftEye,rightEye,nose])


# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 2
imodoE = 1

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

mouthT = np.float32([[1,0,0],[0,1,0]])
leyeT = np.float32([[1,0,0],[0,1,0]])
reyeT = np.float32([[1,0,0],[0,1,0]])
noseT = np.float32([[1,0,0],[0,1,0]])

containersT = [mouthT,leyeT,reyeT,noseT]

px = py = 0
lmb_mantenido = False

mouth_container = [0,0,0,0]
leye_container = [0,0,0,0]
reye_container = [0,0,0,0]
nose_container = [0,0,0,0]

containers = [mouth_container,leye_container,reye_container,nose_container]
active_container = None
active_container_p = [0,0]
z_index = [0,1,2,3]

lanzado = 0

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    if ret:
        if lanzado == 0:
            cv2.imshow('Cam', frame)
            cv2.setMouseCallback('Cam', mouse_events)
            lanzado = 1

        # Search face with a specific setup for face and eye detection
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
        if values is not None:
            face, eyes, shape = values

            [x, y , w, h] = face
            if x > -1:
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                [lex, ley, rex, rey] = eyes
                if lex > -1:
                    setAllFeatureContainers(shape)
                    (masks,features) = generateAllMasksAndFeatures(frame,shape)
                    [mouth_mask,leye_mask,reye_mask,nose_mask] = masks
                    [mouth,leftEye,rightEye,nose] = features

                    full_mask = cv2.bitwise_or(cv2.bitwise_or(cv2.bitwise_or(mouth_mask,leye_mask),reye_mask),nose_mask)
                    face_mask = 255-full_mask
                    frame = cv2.bitwise_and(frame,frame,mask=face_mask)
                    full_mask = cv2.dilate(full_mask, np.ones((3, 3), np.uint8))
                    frame = cv2.inpaint(frame, full_mask, 5, cv2.INPAINT_TELEA)

                    if active_container is not None:
                        dx = active_container_p[0] - containers[active_container][0] + pdx
                        dy = active_container_p[1] - containers[active_container][1] + pdy
                        left_dx_limit = x - containers[active_container][0]
                        right_dx_limit = x + w - containers[active_container][2]
                        top_dy_limit = y - containers[active_container][1]
                        bottom_dy_limit = y + h - containers[active_container][3]
                        containersT[active_container][0][2] = min(max(dx, left_dx_limit), right_dx_limit)
                        containersT[active_container][1][2] = min(max(dy, top_dy_limit), bottom_dy_limit)

                    mouth = cv2.warpAffine(mouth,mouthT,(mouth.shape[1],mouth.shape[0]))
                    leftEye = cv2.warpAffine(leftEye,leyeT,(leftEye.shape[1],leftEye.shape[0]))
                    rightEye = cv2.warpAffine(rightEye,reyeT,(rightEye.shape[1],rightEye.shape[0]))
                    nose = cv2.warpAffine(nose,noseT,(nose.shape[1],nose.shape[0]))

                    features = [mouth,leftEye,rightEye,nose]
                    masks = [cv2.inRange(cv2.cvtColor(feature,cv2.COLOR_BGR2GRAY),1,255) for feature in features]

                    composed_features = np.zeros(frame.shape,dtype=np.uint8)
                    composed_masks = np.zeros(frame.shape[:2],dtype=np.uint8)
                    
                    for i in z_index:
                        feature = cv2.bitwise_and(features[i],features[i],mask=255-composed_masks)
                        composed_features = cv2.bitwise_or(composed_features,feature)
                        composed_masks = cv2.bitwise_or(composed_masks,masks[i])

                    frame = cv2.bitwise_and(frame,frame,mask=255-composed_masks)
                    frame = cv2.bitwise_or(frame,composed_features)
        cv2.imshow('Cam', frame)
        
        # Esc to finish
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc
            break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
